In [1]:
import xarray as xr
import numpy as np

from utils import (
    detrend, monthly_anomalies, reg_slope,corr_with_ttest
)

In [2]:
t_grid_file = xr.open_zarr('data/ORCA025.L46-KFS006_TroPA.zarr')

In [3]:
eq = dict(lat=0, lon=slice(140,260))

#### 1.1 D20 

In [4]:
d20_eq = monthly_anomalies(
     t_grid_file.d20.sel(**eq)
).fillna(0.).compute()
d20_eq.attrs.update({"long_name": "D20 anomaly", "units": "m", "positive": "down"})

#### 1.2 SST

In [5]:
sst_eq = monthly_anomalies(
    t_grid_file.sosstsst.sel(**eq)
).rename("SST")

#### 1.3 SSH

In [6]:
ssh_eq = monthly_anomalies(
    t_grid_file.sossheig.sel(**eq))

ssh_eq.attrs.update({"long_name": "SSH anomaly", "units": "m"})

In [7]:
ds = xr.Dataset({"SST": sst_eq, "D20": d20_eq, "SSH": ssh_eq})

In [8]:
ds_dt = detrend(
    ds.rolling(time=21*12,center=True).construct("window_dim").dropna('time').compute(),
    dim='window_dim',
)

#### 2.1 Figure 8a: Correlation(SST,D20)

In [9]:
corr, c_low, c_up = corr_with_ttest(
    ds_dt.SST, ds_dt.D20, dim="window_dim"
)

In [10]:
non_sig = xr.where(np.abs(corr) <= c_up, 1., np.nan)

#### 2.2 Figure 8b:21y-running regression coefficient 

In [11]:
slope = reg_slope(ds_dt.D20, ds_dt.SSH, dim="window_dim")

In [12]:
corr_ssh_d20, c_low_ssh_d20, c_up_ssh_d20 = corr_with_ttest(
    ds_dt.SSH, ds_dt.D20, dim="window_dim"
)

In [13]:
non_sig_ssh_d20 = xr.where(np.abs(corr_ssh_d20) <= c_up, 1., np.nan)

## Write output

In [14]:
!rm -rf data_fig9.zarr

xr.Dataset(
    dict(
        corr=corr,
        non_sig=non_sig,
        slope=slope,     
    )
).to_zarr('data_fig9.zarr');